# 評価メトリクスの調査

パイプラインの完了後、出来上がったモデルの評価結果を調査します。
生成された evaluation.json をS3からダウンロードし、レポートの中身を表示しましょう。

[evaluate_model_metrics.py](evaluate_model_metrics.py) を参照してください。

In [ ]:
from botocore.exceptions import ClientError

import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

import botocore.config

config = botocore.config.Config(
    user_agent_extra='dsoaws/1.0'
)

sm = boto3.Session().client(service_name="sagemaker", 
                            region_name=region,
                            config=config)

In [ ]:
%store -r pipeline_name

In [ ]:
print(pipeline_name)

In [ ]:
%%time

import time
from pprint import pprint

executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

while pipeline_execution_status == "Executing":
    try:
        executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
        pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
    except Exception as e:
        print("Please wait...")
        time.sleep(30)

pprint(executions_response)

# パイプライン実行ステップを表示

In [ ]:
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

In [ ]:
pipeline_execution_arn = executions_response[0]["PipelineExecutionArn"]
print(pipeline_execution_arn)

In [ ]:
from pprint import pprint

steps = sm.list_pipeline_execution_steps(PipelineExecutionArn=pipeline_execution_arn)

pprint(steps)

# 評価メトリクスを取得

In [ ]:
# for execution_step in reversed(execution.list_steps()):
for execution_step in reversed(steps["PipelineExecutionSteps"]):
    if execution_step["StepName"] == "EvaluateModel":
        processing_job_name = execution_step["Metadata"]["ProcessingJob"]["Arn"].split("/")[-1]

describe_evaluation_processing_job_response = sm.describe_processing_job(ProcessingJobName=processing_job_name)

evaluation_metrics_s3_uri = describe_evaluation_processing_job_response["ProcessingOutputConfig"]["Outputs"][0][
    "S3Output"
]["S3Uri"]

print(evaluation_metrics_s3_uri)

## テスト正確度（accuracy）を表示

In [ ]:
import json
from pprint import pprint

evaluation_json = sagemaker.s3.S3Downloader.read_file("{}/evaluation.json".format(evaluation_metrics_s3_uri))

pprint(json.loads(evaluation_json))

## モデル評価時に生成された混同行列を表示

In [ ]:
!aws s3 cp $evaluation_metrics_s3_uri/confusion_matrix.png ./model_evaluation/

import time

time.sleep(10)  # Slight delay for our notebook to recognize the newly-downloaded file

In [ ]:
%%html

<img src='./model_evaluation/confusion_matrix.png'>

# トレーニングしたモデルのS3からのダウンロードと分析

In [ ]:
training_job_arn = None

for execution_step in steps["PipelineExecutionSteps"]:
    if execution_step["StepName"] == "Train":
        training_job_arn = execution_step["Metadata"]["TrainingJob"]["Arn"]

        break

training_job_name = training_job_arn.split("/")[-1]
print(training_job_name)

In [ ]:
model_tar_s3_uri = sm.describe_training_job(TrainingJobName=training_job_name)["ModelArtifacts"]["S3ModelArtifacts"]

In [ ]:
!aws s3 cp $model_tar_s3_uri ./

In [ ]:
!mkdir -p ./model
!tar -zxvf model.tar.gz -C ./model

# パイプラインで生成されたすべてのアーティファクトを表示

Amazon SageMaker ML Lineage Trackingは、データ準備からモデルデプロイといった機械学習（ML）ワークフローのステップに関する情報を作成・保存します。

Amazon SageMaker Lineageを用いると、SageMaker内で発生したイベントをグラフ構造を介して追跡することができます。
このデータにより、レポートの生成、またはイベント間の関係の比較や発見を行うことができます。
例えば、モデルがどのように生成されたか、モデルがどこにデプロイされたかを簡単に追跡できます。

リネージグラフはSageMakerによって自動的に作成されますが、独自のグラフを直接作成または修正することもできます。

## キーコンセプト

* **リネージグラフ（Lineage Graph）** - 機械学習のワークフローをエンドツーエンドで追跡する連結グラフ。

* **アーティファクト（Artifact）** - URIアドレス指定可能なオブジェクトまたはデータを表す。アーティファクトは通常、アクションへの入力または出力です。

* **アクション（Action）** - 計算、変換、ジョブなどのアクションを表す。

* **コンテキスト（Context）** - エンティティ達を論理的にグループ化する方法を提供する。

* **関連付け（Association）** - 2つのエンティティをリンクするリネージグラフの有向エッジ。

* **リネージ横断性（Lineage Traversal）** - 任意のポイントからスタートしてリネージグラフをトレースし、ワークフローのステップ間の関係を発見、分析することができる。

In [ ]:
processing_job_name = None
training_job_name = None

In [ ]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer

viz = LineageTableVisualizer(sagemaker.session.Session())

for execution_step in reversed(steps["PipelineExecutionSteps"]):
    print(execution_step)
    # We are doing this because there appears to be a bug of this LineageTableVisualizer handling the Processing Step
    if execution_step["StepName"] == "Processing":
        processing_job_name = execution_step["Metadata"]["ProcessingJob"]["Arn"].split("/")[-1]
        print(processing_job_name)
        display(viz.show(processing_job_name=processing_job_name))
    elif execution_step["StepName"] == "Train":
        training_job_name = execution_step["Metadata"]["TrainingJob"]["Arn"].split("/")[-1]
        print(training_job_name)
        display(viz.show(training_job_name=training_job_name))
    else:
        display(viz.show(pipeline_execution_step=execution_step))
        time.sleep(5)

# リソースの解放

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>